In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.model_selection import train_test_split

# Define paths
DATASET_DIR = '/home/user/cnn/data'
CATEGORIES = ['met', 'nichtmet']
IMG_SIZE = 640  # Ensure your system has enough memory for this size
NEW_MODEL_PATH = '/home/user/cnn/model/metal_classifier_model_mobile.keras'
RESULTS_DIR = '/home/user/cnn/results_' + datetime.now().strftime('%Y%m%d_%H%M%S')

os.makedirs(RESULTS_DIR, exist_ok=True)

# Load and preprocess new dataset
def load_data(dataset_dir, categories, img_size):
    data = []
    for category in categories:
        path = os.path.join(dataset_dir, category)
        class_num = categories.index(category)
        for img in os.listdir(path):
            try:
                img_path = os.path.join(path, img)
                img_array = cv2.imread(img_path)
                img_array = cv2.resize(img_array, (img_size, img_size))
                data.append([img_array, class_num])
            except Exception as e:
                print(f"Error loading image {img_path}: {e}")
    return data

data = load_data(DATASET_DIR, CATEGORIES, IMG_SIZE)
np.random.shuffle(data)

X = np.array([item[0] for item in data]) / 255.0
y = np.array([item[1] for item in data])

y = tf.keras.utils.to_categorical(y, num_classes=len(CATEGORIES))

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen.fit(X_train)

# Load MobileNetV2 pre-trained model
base_model = tf.keras.applications.MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3),
                                               include_top=False,
                                               weights='imagenet')

# Freeze the base model
base_model.trainable = False

# Add custom layers on top of base model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(CATEGORIES), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint(os.path.join(RESULTS_DIR, 'best_model.keras'), save_best_only=True, monitor='val_loss', mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)

# Reduce the batch size if you encounter memory issues
batch_size = 16

# Train the model
history = model.fit(datagen.flow(X_train, y_train, batch_size=batch_size),
                    epochs=100,
                    validation_data=(X_val, y_val),
                    callbacks=[early_stopping, checkpoint, reduce_lr])

# Load the best model
model.load_weights(os.path.join(RESULTS_DIR, 'best_model.keras'))

# Fine-tune the model (unfreeze some layers)
base_model.trainable = True

# Recompile the model
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),  # Low learning rate
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Continue training
history_finetune = model.fit(datagen.flow(X_train, y_train, batch_size=batch_size),
                             epochs=100,
                             validation_data=(X_val, y_val),
                             callbacks=[early_stopping, checkpoint, reduce_lr])

# Save the final model
model.save(NEW_MODEL_PATH)

# Save training information
def save_training_info(history, results_dir, phase="initial"):
    # Plot training & validation accuracy and loss
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title(f'Model Accuracy ({phase} phase)')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.savefig(os.path.join(results_dir, f'{phase}_accuracy_plot.png'))

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(f'Model Loss ({phase} phase)')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.savefig(os.path.join(results_dir, f'{phase}_loss_plot.png'))

    # Save history as text
    with open(os.path.join(results_dir, f'{phase}_training_history.txt'), 'w') as f:
        f.write(str(history.history))

save_training_info(history, RESULTS_DIR, phase="initial")
save_training_info(history_finetune, RESULTS_DIR, phase="finetune")

print(f"New model saved to {NEW_MODEL_PATH}")
print(f"Training information saved to {RESULTS_DIR}")


## Additional Code for Test Data Evaluation and Deployment:

## 1. Evaluate Model on Test Data:

In [11]:
# Load test data
# Note: Load test data here from another folder
TEST_DIR = 'F:\\Study\\TU Dortmund\\Industrial Data Science 2\\Code\\Reference\\CNN\\train'
test_data = load_data(TEST_DIR, CATEGORIES, IMG_SIZE)

X_test = np.array([item[0] for item in test_data]) / 255.0
y_test = np.array([item[1] for item in test_data])

y_test = tf.keras.utils.to_categorical(y_test, num_classes=len(CATEGORIES))

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")


21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step - accuracy: 0.9486 - loss: 0.1352
Test Accuracy: 94.09%
Test Loss: 0.1577


## 2. Convert Model for Mobile Deployment:

In [ ]:
# Convert Keras model to SavedModel format
model.save('F:\Study\TU Dortmund\Industrial Data Science 2\Code\Reference\CNN\results_20240702_155253\best_model.keras')

# Convert SavedModel to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_saved_model('saved_model')
tflite_model = converter.convert()

TFLITE_MODEL_PATH = 'metal_classifier_model_mobile.tflite'
with open(TFLITE_MODEL_PATH, 'wb') as f:
    f.write(tflite_model)

print(f"Model converted and saved to {TFLITE_MODEL_PATH}")


In [ ]:
import tensorflow as tf

# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model
TFLITE_MODEL_PATH = 'F:\\Study\\TU Dortmund\\Industrial Data Science 2\\Code\\Reference\\CNN\\metal_classifier_model_mobile.tflite'
with open(TFLITE_MODEL_PATH, 'wb') as f:
    f.write(tflite_model)

print(f"Model converted and saved to {TFLITE_MODEL_PATH}")


## 3. Load and Test the TensorFlow Lite Model:

In [ ]:
# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path=TFLITE_MODEL_PATH)
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Prepare input data
input_data = np.expand_dims(X_test[0], axis=0).astype(np.float32)  # Example with the first test image

# Set the tensor to point to the input data to be inferred
interpreter.set_tensor(input_details[0]['index'], input_data)

# Run the inference
interpreter.invoke()

# Get the results
output_data = interpreter.get_tensor(output_details[0]['index'])
predicted_label = np.argmax(output_data)
print(f"Predicted label: {CATEGORIES[predicted_label]}")


## More improved enhanancements

In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Define paths
DATASET_DIR = '/home/user/cnn/data'
CATEGORIES = ['met', 'nichtmet']
IMG_SIZE = 128
NEW_MODEL_PATH = '/home/user/cnn/model/metal_classifier_model_mobile.keras'
RESULTS_DIR = 'results_' + datetime.now().strftime('%Y%m%d_%H%M%S')

os.makedirs(RESULTS_DIR, exist_ok=True)

# Load and preprocess new dataset
def load_data(dataset_dir, categories, img_size):
    data = []
    for category in categories:
        path = os.path.join(dataset_dir, category)
        class_num = categories.index(category)
        for img in os.listdir(path):
            try:
                img_path = os.path.join(path, img)
                img_array = cv2.imread(img_path)
                img_array = cv2.resize(img_array, (img_size, img_size))
                data.append([img_array, class_num])
            except Exception as e:
                print(f"Error loading image {img_path}: {e}")
    return data

data = load_data(DATASET_DIR, CATEGORIES, IMG_SIZE)
np.random.shuffle(data)

X = np.array([item[0] for item in data]) / 255.0
y = np.array([item[1] for item in data])

y = tf.keras.utils.to_categorical(y, num_classes=len(CATEGORIES))

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    brightness_range=[0.8, 1.2],
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen.fit(X_train)

# Load MobileNetV2 pre-trained model
base_model = tf.keras.applications.MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3),
                                               include_top=False,
                                               weights='imagenet')

# Add custom layers on top of base model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(len(CATEGORIES), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint(os.path.join(RESULTS_DIR, 'best_model.keras'), save_best_only=True, monitor='val_loss', mode='min')

# Train the model
history = model.fit(datagen.flow(X_train, y_train, batch_size=64),
                    epochs=50,
                    validation_data=(X_val, y_val),
                    callbacks=[early_stopping, checkpoint])

# Evaluate on validation set
val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
print(f'Final Validation Loss: {val_loss:.4f}, Final Validation Accuracy: {val_acc:.4f}')

# Print classification report
y_pred = model.predict(X_val)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_val, axis=1)
print(classification_report(y_true, y_pred_classes, target_names=CATEGORIES))

# Save the final model
model.save(NEW_MODEL_PATH)

# Save training information
def save_training_info(history, results_dir, phase="initial"):
    # Plot training & validation accuracy and loss
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title(f'Model Accuracy ({phase} phase)')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.savefig(os.path.join(results_dir, f'{phase}_accuracy_plot.png'))

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(f'Model Loss ({phase} phase)')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.savefig(os.path.join(results_dir, f'{phase}_loss_plot.png'))

    # Save history as text
    with open(os.path.join(results_dir, f'{phase}_training_history.txt'), 'w') as f:
        f.write(str(history.history))

save_training_info(history, RESULTS_DIR, phase="initial")

print(f"New model saved to {NEW_MODEL_PATH}")
print(f"Training information saved to {RESULTS_DIR}")


In [ ]:
# Evaluation metrics
from sklearn.metrics import classification_report

# Load the best model for evaluation
model = tf.keras.models.load_model('F:\\Study\\TU Dortmund\\Industrial Data Science 2\\Code\\Reference\\CNN\\results_20240702_163204\\best_model.keras')

# Evaluate on validation set
val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}')

# Get predictions for confusion matrix and classification report
y_pred = model.predict(X_val)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_val, axis=1)

# Print classification report
print(classification_report(y_true, y_pred_classes, target_names=CATEGORIES))


In [ ]:
# Evaluate on validation set
val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
print(f'Final Validation Loss: {val_loss:.4f}, Final Validation Accuracy: {val_acc:.4f}')

# Print classification report
y_pred = model.predict(X_val)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_val, axis=1)
print(classification_report(y_true, y_pred_classes, target_names=CATEGORIES))


### Train new for mobile images 

In [ ]:
import os
import numpy as np
import cv2
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import classification_report, precision_recall_curve, average_precision_score

# Enable mixed precision
tf.keras.mixed_precision.set_global_policy('mixed_float16')

# Define paths
DATASET_DIR = '/home/user/unet/cnn/data'
CATEGORIES = ['met', 'nichtmet']
IMG_SIZE = 512  # Reduced image size

# Function to load and preprocess the images
def load_data():
    data = []
    for category in CATEGORIES:
        path = os.path.join(DATASET_DIR, category)
        class_num = CATEGORIES.index(category)
        for img in tqdm(os.listdir(path)):
            try:
                img_path = os.path.join(path, img)
                img_array = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                img_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                data.append([img_array, class_num])
            except Exception as e:
                pass
    return data

# Load and shuffle data
data = load_data()
np.random.shuffle(data)

# Split data into features (X) and labels (y)
X = np.array([item[0] for item in data]).reshape(-1, IMG_SIZE, IMG_SIZE, 1) / 255.0
y = np.array([item[1] for item in data])

# One-hot encode the labels
y = to_categorical(y, num_classes=len(CATEGORIES))

# Split data into training, validation, and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

# Fit the data generator to the training data
datagen.fit(X_train)

# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Flatten(),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),

    Dense(len(CATEGORIES), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Model checkpoint callback
checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss', mode='min')

# Reduce learning rate callback
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=1e-4)

# Train the model with batch size 2
history = model.fit(datagen.flow(X_train, y_train, batch_size=2),
                    epochs=100,
                    validation_data=(X_val, y_val),
                    callbacks=[early_stopping, checkpoint, reduce_lr])

# Load the best model
model.load_weights('best_model.keras')

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Predict on test data
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Classification report
report = classification_report(y_true_classes, y_pred_classes, target_names=CATEGORIES)
print(report)

# Precision and recall graph
precision = dict()
recall = dict()
average_precision = dict()
for i in range(len(CATEGORIES)):
    precision[i], recall[i], _ = precision_recall_curve(y_test[:, i], y_pred[:, i])
    average_precision[i] = average_precision_score(y_test[:, i], y_pred[:, i])

plt.figure()
for i in range(len(CATEGORIES)):
    plt.plot(recall[i], precision[i], lw=2, label=f'Precision-recall curve of class {CATEGORIES[i]} (AP={average_precision[i]:0.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc='best')
plt.show()

# Plot training & validation accuracy and loss
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.show()

# Save the model
model.save('metal_classifier_model_mobile_from_scratch.keras')
print("Model saved to metal_classifier_model_mobile_from_scratch.keras")
